# 構築

クラウド上でLinuxマシンを起動して、Moodleを構築します。

## ユーザ識別のため、ユーザ名を設定

In [ ]:
username = 'lc4ri099'

## awscliの準備

### インストール

In [ ]:
!sudo apt-get update && sudo apt-get upgrade -y && sudo apt-get install -y awscli

In [ ]:
!aws --version

### 設定

In [ ]:
target_region = 'ap-northeast-1'
!aws configure set default.region {target_region}
!aws configure set default.output json

認証情報としてAccessKeyIdとSecretAccessKeyを設定します。

コマンドに対する対話的な操作が必要ですから、JupyterのTerminal(treeページの[New] - [Terminal]から選択できます)から、 `aws configure` を実施してください。

`aws configure` の実施例:

```
$ aws configure
AWS Access Key ID [None]: (自身のアカウントのアクセスキー)
AWS Secret Access Key [None]: (自身のアカウントのシークレットアクセスキー)
Default region name [ap-northeast-1]: (Enter)
Default output format [json]: (Enter)
```

### boto3ライブラリの追加

ansibleからawsを触るために、ライブラリを追加しておきます。

In [ ]:
!conda install -y boto3 boto

## keypairの作成

クラウド上でLinuxマシンを起動する前に、ssh接続に必要となるkeypairを作成します。

In [ ]:
!aws ec2 delete-key-pair --key-name {username}
row = !aws ec2 create-key-pair --key-name {username} --query 'KeyMaterial' --output text

with open(username + '.pem', mode='w') as f:
    f.write('\n'.join(row))

import os
os.chmod(username + '.pem', 0o600)

#!aws ec2 delete-key-pair --key-name {username}

ansibleでもec2_keyモジュールで同じ操作が行なえます。

```
import json

!ansible localhost -m ec2_key -a "name={username} state=absent"

row = !ansible localhost -m ec2_key -a "name={username} region={target_region} profile=default"

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
k = json.loads(''.join(row))
with open(username + '.pem', mode='w') as f:
    f.write(k["key"]["private_key"])

import os
os.chmod(username + '.pem', 0o600)

#!ansible localhost -m ec2_key -a "name={username} state=absent"
```

## Linuxマシンを起動する

ansibleでec2インスタンスを起動します。ディストリビューションはUbuntuとします。

In [ ]:
image = 'ami-0eeb679d57500a06c' # Ubuntu Server 18.04 LTS (HVM), SSD Volume Type
instance_type = 't3.medium'
subnet = 'subnet-8a6618d1' # 先に作成済みのもの
sg = 'seminar-lc4ri' # 先に作成済みのもの

import json

row = !ansible localhost -m ec2 -a "region={target_region} key_name={username} \
profile=default wait=yes assign_public_ip=yes \
image={image} instance_type={instance_type} vpc_subnet_id={subnet} group={sg}"

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
linux = json.loads(''.join(row))
print(linux)

起動が完了するまで待ちます。

In [ ]:
!aws ec2 wait instance-status-ok --include-all-instances --instance-ids {linux['instance_ids'][0]}

起動したLinuxマシンでOSアップデートを実行します。

まず、接続情報を作成します。

In [ ]:
host = linux["instances"][0]["public_ip"]
ansible_user = "ubuntu"
ansible_ssh_private_key_file = username + ".pem"

with open('hosts', mode='w') as f:
    f.write(host + ' ansible_user=' + ansible_user + ' ansible_ssh_private_key_file='
            + ansible_ssh_private_key_file)

作成した接続情報を使う前に、対象となるLinuxマシンを信頼できるホストとして、fingerprint登録しておきます。JupyterのTerminalを使って、ダミーのssh実行を行うと登録されます。

```
$ ssh a@（対象となるLinuxマシンのIPアドレス）
（中略）
Are you sure you want to continue connecting (yes/no)? yes←信頼しますよの意味で入力する
（以下略）
```

ssh接続を成功させたいわけではないので、接続自体は失敗させてしまって構いません。

実務では、ansibleの対象となるLinuxマシンが大量だったり不定だったりするかも知れません。その場合は、sshやansibleの設定で、fingerprint登録の有無を確認しないこともできます。

ansibleでaptを実行します。

In [ ]:
!ansible -i hosts all -m apt -a "name=* state=latest update_cache=yes" --become

## Docker環境を用意する

[moodle-docker: Docker Containers for Moodle Developers](https://github.com/moodlehq/moodle-docker)を参考に作業します。

docker-composeが必要なので、インストールします。Ubuntuでは、パッケージで入れることができて、依存関係から、dockerも入ります。

In [ ]:
!ansible -i hosts all -m apt -a "name=docker-compose" --become

パッケージ類を一通り入れ終わったので、rebootしておきます。（Ubuntuはkernelが頻繁に上がるので）

```
!ansible localhost -m ec2 -a "region={target_region} key_name={username} \
profile=default instance_ids={linux['instance_ids'][0]} state=restarted"
```

でrebootを指示できるのですが、OSが上がってssh可能となるタイミングが分からない（たぶん5分放置とかでいけるような気はしますが）ので、
terminalからsshして作業することにします。

```
$ ssh -i lc4rixxx.pem ubuntu@xxx.xxx.xxx.xxx
```

Linuxマシンに入ったら、rebootします。

```
$ sudo reboot
```

しばらくしてから、sshで接続を試し、OSが上がってssh可能となったことが確認できたら、次に進みます。

## Moodleのコードをダウンロードする

Moodle本体のコードと、dockerで起動するためのコードを、gitでダウンロードします。

In [ ]:
!ansible -i hosts all -m git -a "repo=git://git.moodle.org/moodle.git version=MOODLE_37_STABLE \
depth=1 dest=/home/ubuntu/moodle"

In [ ]:
!ansible -i hosts all -m git -a "repo=https://github.com/moodlehq/moodle-docker.git \
depth=1 dest=/home/ubuntu/moodle-docker"

moodle-dockerに含まれるMoodleの設定ファイルをmoodleにコピーします。

In [ ]:
!ansible -i hosts all -m shell -a "chdir=/home/ubuntu/moodle-docker \
cp config.docker-template.php /home/ubuntu/moodle/config.php"

## 起動とwebインストール

In [ ]:
!ansible -i hosts all -m shell -a "chdir=/home/ubuntu/moodle-docker \
MOODLE_DOCKER_WWWROOT=/home/ubuntu/moodle \
MOODLE_DOCKER_DB=pgsql \
MOODLE_DOCKER_WEB_HOST={host} \
bin/moodle-docker-compose up -d" --become

In [ ]:
!ansible -i hosts all -m shell -a "chdir=/home/ubuntu/moodle-docker \
MOODLE_DOCKER_WWWROOT=/home/ubuntu/moodle \
MOODLE_DOCKER_DB=pgsql \
MOODLE_DOCKER_WEB_HOST={host} \
bin/moodle-docker-wait-for-db" --become

起動したMoodleは、Linuxマシンの8000番ポートからサービスされているので、ブラウザでアクセスし、webインストールを行います。

## 停止

DBを削除するので、webインストール以降の内容は失われますので、注意してください。

In [ ]:
!ansible -i hosts all -m shell -a "chdir=/home/ubuntu/moodle-docker \
MOODLE_DOCKER_WWWROOT=/home/ubuntu/moodle \
MOODLE_DOCKER_DB=pgsql \
MOODLE_DOCKER_WEB_HOST={host} \
bin/moodle-docker-compose down" --become